### Setup

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans, whiten

In [23]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [25]:
undata = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/curric-data-001-data-sets/master/un/un.csv')
undata.head()

,country,region,tfr,contraception,educationMale,educationFemale,lifeMale,lifeFemale,infantMortality,GDPperCapita,economicActivityMale,economicActivityFemale,illiteracyMale,illiteracyFemale
0,Afghanistan,Asia,6.90,NaN,NaN,NaN,45.0,46.0,154.0,2848.0,87.5,7.2,52.800,85.00
1,Albania,Europe,2.60,NaN,NaN,NaN,68.0,74.0,32.0,863.0,NaN,NaN,NaN,NaN
2,Algeria,Africa,3.81,52.0,11.1,9.9,67.5,70.3,44.0,1531.0,76.4,7.8,26.100,51.00
3,American.Samoa,Asia,NaN,NaN,NaN,NaN,68.0,73.0,11.0,NaN,58.8,42.4,0.264,0.36
4,Andorra,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
undata.count()

country                   207
region                    207
tfr                       197
contraception             144
educationMale              76
educationFemale            76
lifeMale                  196
lifeFemale                196
infantMortality           201
GDPperCapita              197
economicActivityMale      165
economicActivityFemale    165
illiteracyMale            160
illiteracyFemale          160
dtype: int64

In [58]:
undata.dropna(inplace = True)
x = undata[['lifeMale', 'lifeFemale', 'infantMortality']]
print(x)

     lifeMale  lifeFemale  infantMortality
2        67.5        70.3             44.0
12       70.5        77.1             14.0
13       71.1        75.3             18.0
18       73.4        76.1             30.0
23       48.9        51.7             56.0
27       45.1        47.0             97.0
28       45.5        48.8            114.0
35       72.3        78.3             13.0
43       74.2        78.0              9.0
50       68.9        73.1             34.0
53       64.7        67.3             54.0
54       66.5        72.5             39.0
57       63.9        75.0             12.0
65       45.4        48.7            122.0
67       68.5        76.7             23.0
80       76.1        81.8              5.0
81       64.5        73.8             14.0
84       63.3        67.0             48.0
86       60.9        63.9             95.0
91       72.4        76.8             12.0
98       68.8        76.0              9.0
99       74.1        78.2             14.0
102      62

In [68]:
codebooks = []
distortions = []
ks = []
features = whiten(x)
for k in range(1,9):
    ks.append(k)
    codebook, distortion = kmeans(features,k)
    codebooks.append(codebook)
    distortions.append(distortion)
means = np.column_stack([ks, distortions])
# means
sns.pointplot(means)
# plt.show()

IndexError: arrays used as indices must be of integer (or boolean) type